### 📘 Introduction to PyPSA

<div style="display: flex; align-items: center; justify-content: space-between;">
  <div>
    <h3>Course presenters</h3>
    <ul>
      <li><strong>Name Surname</strong> - Role</li>
      <li><strong>Name Surname</strong> - Role</li>
    </ul>
  </div>
  <div>
    <a href="https://openenergytransition.org/index.html">
      <img src="https://openenergytransition.org/assets/img/oet-logo-red-n-subtitle.png" height="60" alt="OET">
    </a>
  </div>
</div>


##### 🎯 Learning Objectives  



* Introduce participants to the PyPSA toolbox. 
* Import a PyPSA network. 
* Provide details of relevant components.  
* Solve a simple PyPSA model.  
* Review the data structures for static and time-series data.  
* Analyze the results.  
---

### 📄 **Case Study**

_Provide the context of the scenario._

In this case we have a model of eight countries in Europe for a 24 hour period. We want to observe the results of this model. 


### 📥 **Importing Essential Libraries**  


In [ ]:
# Google Colab users
# Remove the comments in the rows below to set up your notebook.

# !pip install pypsa

# from google.colab import drive
# import os

# drive.mount('/content/drive')
# os.chdir('/content/drive/MyDrive/psfo_2025/mec4131z/')

In [ ]:
import pypsa
import pandas as pd


#### 📥 **Importing Networks in PyPSA** 

## 📅 Importing Networks in PyPSA

PyPSA supports importing networks in **CSV**, **XLSX**, **HDF5**, and **NetCDF** formats. The table below summarizes the common formats and how to import them:

| Format       | Icon | Description                                          | Import Example                                                   |
|--------------|------|------------------------------------------------------|------------------------------------------------------------------|
| `CSV` Folder   | 📂   | Readable, editable input data                        | `network.import_from_csv_folder("path_to_csv_directory")`       |
| `XLSX` | 📄   | Small networks and spreadsheet editing | `network.import_from_excel("path_to_file.xlsx")`                |
| `HDF5`         | 💾   | Fast and efficient binary format                      | `network = pypsa.Network("path_to_file.h5")`                    |
| `NetCDF`       | 📆   | Common for scientific computing                       | `network = pypsa.Network("path_to_file.nc")`                    |


⤴️ **Shortcut**

Instead of using the `network.import_from_...("file_path/file_name.xxx")` function to import a network, you can simply add the path and file location in the Network function and PyPSA will select the appropriate method.


`network = pypsa.Network("file_path/file_name.xxx")`

### **🔧⚡ Create a PyPSA Network Object**

In [ ]:
network = pypsa.Network('EU_Test.nc')

Look at what is in the network.

In [ ]:
network.all_components

Look at the component attributes.

In [ ]:
network.component_attrs['Generator']

In [ ]:
network.component_attrs['Generator'].head()

Each component is also given a label `list_name` which is the label used to access the dataset. 

In [ ]:
network.components

In [ ]:
for key in network.component_attrs:
    print(f'{key.ljust(20)} {network.components[key]["list_name"]}')


#### 📂 Data Structure Guidelines  



✅ **Static Data:**  
- The Excel file should be configured using the `list_name` as the `sheet name`.  
- Use ``variables`` in the header.  

✅ **Time-Series Data:**  
- Follow the naming convention:  `[list]-[variable name]`
- Example: `loads-p_set` for the set loads.

---


#### ⏳ Snapshots


A `snapshot` represents a specific point in time for which the network is simulated.  
- Snapshots can be single timestamps (e.g., `2025-01-01 00:00`) or time series covering hours, days, weeks, or years.  
- They allow modeling of dynamic power system behavior over different time periods.  

In [ ]:
network.snapshots

#### 🔌 Carrier

A carrier defines the type of energy being transported or converted in the network. By default, it is set to "AC" for alternating current electricity networks, but it can be set to "DC" or any custom value such as "wind", "heat", "hydrogen", or "gas".

Carriers can also store attributes relevant to global constraints—e.g., CO₂ emissions per carrier—for use in system-wide emissions limits or cost calculations.



In [ ]:
network.carriers

#### 🔹 Bus

The **bus** is the fundamental node of the network. Components like generators, loads, and transmission lines connect to it. It ensures energy conservation by enforcing that all inflows and outflows at the bus are balanced—analogous to **Kirchhoff’s Current Law (KCL)**.

In [ ]:
network.buses

#### ⚡ Generators

Generators attach to a single bus, converting energy from their `carrier` to the bus `carrier`.

- Their power output is constrained by `p_nom * p_max_pu` and `p_nom * p_min_pu`.
- Static limits define dispatchable generators, while time-varying limits model renewables.
- Time series `p_max_pu` and `p_min_pu` determine availability per snapshot.
- For unit commitment constraints, refer to the PyPSA documentation.

In [ ]:
network.generators

🌞 **Applying `p_max_pu` Constraint on VRE Generators**  

* Variable Renewable Energy (VRE) generators, such as solar and wind, have time-dependent availability limits.  
* The `p_max_pu` constraint, imported as a time-series dataset, determines the maximum power output at each snapshot on a per unit basis.  


In [ ]:
network.generators_t.p_max_pu.plot()

#### 🔌 Loads

A **load** connects to a single bus and represents energy consumption, such as electricity demand, hydrogen, or heat. It draws active power from the bus and, if reactive power is involved, behaves like an inductor. Loads are essential for modeling demand in power system simulations. ⚡🏠


In [ ]:
network.loads

In [ ]:
network.loads_t.p_set

#### 🔋 Storage options

PyPSA models energy storage using two components: **Storage Units** and **Stores**. A Storage Unit connects to a single bus and shifts power across time using a time-varying state of charge, accounting for charging and discharging efficiencies. Its energy capacity is defined by multiplying `max_hours` with its nominal power.

Stores also connect to a bus but model only the energy balance, not the power flow. They are more flexible for capacity sizing but require additional components like Links to control power in and out. Storage Units offer a more direct and self-contained approach to model battery-like behavior, while Stores are more versatile for complex energy storage systems.

In [ ]:
network.stores

In [ ]:
network.storage_units

🔗 **Links**

* Links enable controllable, directed power flow between two buses (`bus0 → bus1`).  
* They can have efficiency losses and marginal costs, restricting default flow to one direction.  
* For bidirectional, lossless operation, set `efficiency = 1`, `marginal_cost = 0`, and `p_min_pu = -1`.  
* Links model HVDC interconnections, converters, heat pumps, electrolysers, and other controllable power flows.  
* ⚠️ In the actual model, lines will be used instead of links for passive AC/DC transmission.  



In [ ]:
network.links

#### 🌐 Working with the `network` object

The network contains functions, such as: 

- 📥 Adding data: `network.add()` or `network.import_from_csv()` - As described before.
- ✅ `network.consistency_check()` to check network consistency
- 🔍 Optimization: `network.optimize()` – Runs the optimization process.  
   * Supports multiple solvers including GLPK, Gurobi, CPLEX, and HiGHS. 
- 📊 Statistics: `network.statistics()` – Generates system-wide statistics.  
- 🗺️ Visualization: `network.plot()` – Plots the network layout.  


In [ ]:
network.consistency_check()

**Solve Model**

In [ ]:
network.optimize(solver_name='highs')

**Results**

In [ ]:
network.generators_t.p.head() # or .plot()

In [ ]:
network.storage_units_t.p.head() # or .plot()

### 
---